<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/smallmolecules-Pradeep/bioco_smallmolecules_veiledet_regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [ ]:
!pip install optuna

Importering av nødvendig bibliotek og pakker

In [ ]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev
import time

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [ ]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_trening_validering_testing_evaluering.py /content/moduler_trening_validering_testing_evaluering.py

/content


In [ ]:
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_enzym
from moduler_trening_validering_testing_evaluering import trening_testsett_oppdeling_dag
from moduler_trening_validering_testing_evaluering import trening_validering_oppdeling_dag

from moduler_trening_validering_testing_evaluering import enzymfordeling
from moduler_trening_validering_testing_evaluering import n_splitt_enzym
from moduler_trening_validering_testing_evaluering import Resultat_evaluering
from moduler_trening_validering_testing_evaluering import res_eval_opt
from moduler_trening_validering_testing_evaluering import res_eval
from moduler_trening_validering_testing_evaluering import Resultat_dicts
from moduler_trening_validering_testing_evaluering import evaluering_beregning
from moduler_trening_validering_testing_evaluering import enzymtype_rawmaterial_prediksjon

In [ ]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [ ]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [ ]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/smallmolecules_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

# Lagring av resultater

In [ ]:
# Oppretter instans for lagring av resultater
res = Resultat_dicts()

# Parametere til hyperoptimalisering

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
n_gjentagelser = 2

Tilfeldighetsfrø

In [ ]:
#Tilfeldighetsfrø
random_seed = 2024

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

### Statiske estimater med kategorisk informasjon

In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

rskf = RepeatedStratifiedKFold(n_splits=n_splitt,
                                 n_repeats=n_gjentagelser,
                                 random_state=random_seed)

statistisk_res = Resultat_evaluering()

# Går igjennom kryssvalidering
for train, val in rskf.split(trening_markert,
                              enzymtyper_markert):
  # Markerte folds
  trenings_fold_markert = trening_markert.iloc[train, :]
  val_fold = trening_markert.iloc[val, :]

  # Markert respons
  y_trening_fold = trenings_fold_markert.iloc[:, -1]
  y_val_fold = val_fold.iloc[:, -1]


  y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'SmallMolecules', 'mean')
  y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'SmallMolecules', 'mean')
  y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'SmallMolecules', 'mean')


  statistisk_res.evaluering(y_trening_fold,
                            y_trening_pred,
                            y_val_fold,
                            y_val_pred,
                            y_test,
                            y_test_pred)

statistisk_res.beskriv_statistikk()

### VL ML modeller

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt,
                             dict_res=res.dict_res_1_hel,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_hel = pd.DataFrame(res.dict_res_1_hel)
res_1_hel_opt = pd.DataFrame(res.dict_res_1_hel_opt)

res_1_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_hel.csv')
res_1_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_hel_opt.csv')

[I 2024-05-01 21:44:28,762] A new study created in memory with name: no-name-3a083da8-f95b-4f4b-91fd-db9f29cc5f45
[I 2024-05-01 21:44:29,189] Trial 0 finished with value: 0.5931825251653894 and parameters: {'n_neighbors': 11, 'weights': 'distance', 'p': 8}. Best is trial 0 with value: 0.5931825251653894.
[I 2024-05-01 21:44:29,233] Trial 1 finished with value: 0.6126333473618661 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 4}. Best is trial 0 with value: 0.5931825251653894.
[I 2024-05-01 21:44:29,660] Trial 2 finished with value: 0.6106110027390422 and parameters: {'n_neighbors': 15, 'weights': 'distance', 'p': 8}. Best is trial 0 with value: 0.5931825251653894.
[I 2024-05-01 21:44:29,725] Trial 3 finished with value: 0.6074271990708733 and parameters: {'n_neighbors': 15, 'weights': 'distance', 'p': 6}. Best is trial 0 with value: 0.5931825251653894.
[I 2024-05-01 21:44:30,006] Trial 4 finished with value: 0.5996638836262235 and parameters: {'n_neighbors': 2, 'weights

Velger beste modell

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [ ]:
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)


statistisk_res = Resultat_evaluering()
# Går igjennom kryssvalidering
trenings_sett, val_sett\
= trening_validering_oppdeling_dag(rå_data,
                                    trening_markert,
                                    n_splitt=n_splitt,
                                    test_ekskluder_kalkun_kylling=True)

for i, j in zip(trenings_sett, val_sett):
    train = trenings_sett[i]
    val = val_sett[j]

    # Markerte folds
    trenings_fold_markert = trening_markert.loc[train, :]
    val_fold = trening_markert.loc[val, :]

    # Markert respons
    y_trening_fold = trenings_fold_markert.iloc[:, -1]
    y_val_fold = val_fold.iloc[:, -1]


    y_trening_pred, y_trening_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, trenings_fold_markert, 'SmallMolecules', 'mean')
    y_val_pred, y_val_fold = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, val_fold, 'SmallMolecules', 'mean')
    y_test_pred, y_test = enzymtype_rawmaterial_prediksjon(rå_data, trenings_fold_markert, testsett, 'SmallMolecules', 'mean')


    statistisk_res.evaluering(y_trening_fold,
                              y_trening_pred,
                              y_val_fold,
                              y_val_pred,
                              y_test,
                              y_test_pred)

statistisk_res.beskriv_statistikk()

### VL ML modeller

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt,
                             dict_res=res.dict_res_2_hel,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_2_hel = pd.DataFrame(res.dict_res_2_hel)
res_2_hel_opt = pd.DataFrame(res.dict_res_2_hel_opt)

res_2_hel.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_hel.csv')
res_2_hel_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_hel_opt.csv')

[I 2024-05-01 22:14:28,953] A new study created in memory with name: no-name-4fedb9cf-f7f1-403f-a927-1ec50af8666d
[I 2024-05-01 22:14:41,747] Trial 1 finished with value: 0.6755018944356969 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 8}. Best is trial 1 with value: 0.6755018944356969.
[I 2024-05-01 22:14:41,901] Trial 0 finished with value: 0.6461272210176268 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 6}. Best is trial 0 with value: 0.6461272210176268.
[I 2024-05-01 22:14:54,397] Trial 3 finished with value: 0.6817963361970684 and parameters: {'n_neighbors': 18, 'weights': 'uniform', 'p': 2}. Best is trial 0 with value: 0.6461272210176268.
[I 2024-05-01 22:14:54,788] Trial 2 finished with value: 0.6912694134204654 and parameters: {'n_neighbors': 18, 'weights': 'uniform', 'p': 6}. Best is trial 0 with value: 0.6461272210176268.
[I 2024-05-01 22:15:07,269] Trial 4 finished with value: 0.6468357835999183 and parameters: {'n_neighbors': 10, 'weights':

Velger beste modell

## Designproduksjon

## Alternativ 1: 80% Testsett og Hele resterende treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.80
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt,
                             dict_res=res.dict_res_1_design,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

res_1_design = pd.DataFrame(res.dict_res_1_design)
res_1_design_opt = pd.DataFrame(res.dict_res_1_design_opt)

res_1_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_design.csv')
res_1_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_design_opt.csv')

[I 2024-04-28 15:36:41,658] A new study created in memory with name: no-name-13ad8672-a67f-4f70-8389-fa99e7589667
[I 2024-04-28 15:36:42,066] Trial 1 finished with value: 4.287851594475565 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 1}. Best is trial 1 with value: 4.287851594475565.
[I 2024-04-28 15:36:42,097] Trial 0 finished with value: 4.382262579331768 and parameters: {'n_neighbors': 8, 'weights': 'distance', 'p': 10}. Best is trial 1 with value: 4.287851594475565.
[I 2024-04-28 15:36:42,409] Trial 2 finished with value: 4.681101455047659 and parameters: {'n_neighbors': 2, 'weights': 'uniform', 'p': 3}. Best is trial 1 with value: 4.287851594475565.
[I 2024-04-28 15:36:42,462] Trial 3 finished with value: 4.740376919200939 and parameters: {'n_neighbors': 2, 'weights': 'distance', 'p': 4}. Best is trial 1 with value: 4.287851594475565.
[I 2024-04-28 15:36:42,791] Trial 4 finished with value: 4.442448791944665 and parameters: {'n_neighbors': 7, 'weights': 'uniform',

Velger beste modell

## Alternativ 2: 80% Testsett og resteremde hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.80
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

In [ ]:
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt,
                             dict_res=res.dict_res_2_design,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_2_design = pd.DataFrame(res.dict_res_2_design)
res_2_design_opt = pd.DataFrame(res.dict_res_2_design_opt)

res_2_design.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_design.csv')
res_2_design_opt.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_design_opt.csv')

[I 2024-05-02 11:19:16,133] A new study created in memory with name: no-name-919c4f4f-1a88-4b61-a3a2-a79f8408a9a8
[I 2024-05-02 11:19:25,635] Trial 1 finished with value: 0.7512051665624485 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 6}. Best is trial 1 with value: 0.7512051665624485.
[I 2024-05-02 11:19:25,665] Trial 0 finished with value: 0.756044650177284 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'p': 8}. Best is trial 1 with value: 0.7512051665624485.
[I 2024-05-02 11:19:32,496] Trial 3 finished with value: 0.7847857282411089 and parameters: {'n_neighbors': 15, 'weights': 'uniform', 'p': 4}. Best is trial 1 with value: 0.7512051665624485.
[I 2024-05-02 11:19:32,500] Trial 2 finished with value: 0.8578343475504727 and parameters: {'n_neighbors': 1, 'weights': 'distance', 'p': 5}. Best is trial 1 with value: 0.7512051665624485.
[I 2024-05-02 11:19:39,152] Trial 4 finished with value: 0.7913173619993155 and parameters: {'n_neighbors': 2, 'weights': 'un

Velger beste modell

# NIR målinger ekskludert

In [ ]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0

## Hele datasett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_hel_opt_NIR,
                             dict_res=res.dict_res_1_hel_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_hel_NIR = pd.DataFrame(res.dict_res_1_hel_NIR)
res_1_hel_opt_NIR = pd.DataFrame(res.dict_res_1_hel_opt_NIR)

res_1_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_hel_NIR.csv')
res_1_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_hel_opt_NIR.csv')

[I 2024-06-16 19:03:19,402] A new study created in memory with name: no-name-28b09f66-075d-4148-8a7d-5fff9b550d2a
[I 2024-06-16 19:03:20,314] Trial 0 finished with value: 0.8723163906749571 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 4}. Best is trial 0 with value: 0.8723163906749571.
[I 2024-06-16 19:03:20,410] Trial 1 finished with value: 0.8923052469124384 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 10}. Best is trial 0 with value: 0.8723163906749571.
[I 2024-06-16 19:03:21,246] Trial 3 finished with value: 0.8721996045506965 and parameters: {'n_neighbors': 6, 'weights': 'uniform', 'p': 1}. Best is trial 3 with value: 0.8721996045506965.
[I 2024-06-16 19:03:21,412] Trial 2 finished with value: 0.8906024709899892 and parameters: {'n_neighbors': 8, 'weights': 'uniform', 'p': 4}. Best is trial 3 with value: 0.8721996045506965.
[I 2024-06-16 19:03:21,870] Trial 4 pruned. 
[I 2024-06-16 19:03:22,389] Trial 5 finished with value: 0.91866551905085 and pa

Velger beste modell

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_hel_opt_NIR,
                             dict_res=res.dict_res_2_hel_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resulater
res_2_hel_NIR = pd.DataFrame(res.dict_res_2_hel_NIR)
res_2_hel_opt_NIR = pd.DataFrame(res.dict_res_2_hel_opt_NIR)

res_2_hel_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_hel_NIR.csv')
res_2_hel_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_hel_opt_NIR.csv')

Velger beste modell

## Designsett

## Alternativ 1: 20% Testsett og Hele resterende treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, _, _, _, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)
# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(dict_res_opt=res.dict_res_1_design_opt_NIR,
                             dict_res=res.dict_res_1_design_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_gjentagelser=n_gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_1_design_NIR = pd.DataFrame(res.dict_res_1_design_NIR)
res_1_design_opt_NIR = pd.DataFrame(res.dict_res_1_design_opt_NIR)

res_1_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_design_NIR.csv')
res_1_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_1_design_opt_NIR.csv')

Velger beste modell

## Alternativ 2: 20% Testsett og resteremde hele treningssett

In [ ]:
# Deler datasettet i trening og testsett
trening_markert, testsett, _, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 test_ekskluder_kalkun_kylling=True)

# Finner antall av enzymtype med minst antall
n_splitt = n_splitt_enzym(rå_data,
                          trening_markert)

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(dict_res_opt=res.dict_res_2_design_opt_NIR,
                             dict_res=res.dict_res_2_design_NIR,
                             rå_data=rå_data,
                             treningssett_markert=trening_markert,
                             testsett=testsett,
                             n_splitt=n_splitt,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

# Lagring av resultater
res_2_design_NIR = pd.DataFrame(res.dict_res_2_design_NIR)
res_2_design_opt_NIR = pd.DataFrame(res.dict_res_2_design_opt_NIR)

res_2_design_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_design_NIR.csv')
res_2_design_opt_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/Resultater/smallmolecules/smallmolecules_resultater_2_design_opt_NIR.csv')

Velger beste modell